## 시계열데이터 예측/분류 모델(lightGBM)
* Step A. Features Engineering
* Step B. lightGBM modeling
* Step C. Features Reduction
* Step D. Hyperparameter Optimization
* Step E. Save the model

* 참고
* prophet: 

#### Step A. Feature Engineering
  - LightGBM은 tree-based 모델로 scaling을 특별히 요구하지 않음
  - Prediction model 일 때는, 과거 데이터 정보(X)로만 현재 데이터(y)를 Regression 할 수 있도록 함
  - 일반적인 Regression model 일 때는, 타 데이터의 현재 데이터(X)를 사용해도 됨
  - 참고로, LSTM은 (t-k, ..., t-1) ---> t 에서 별도의 feature engineering을 수행하지 않음  

In [ ]:
import numpy as np
import pandas as pd
import holidays
from scipy.fft import fft
from scipy.stats import entropy
import lightgbm as lgb
#from lightgbm import LGBMRegressor  # sckit에서 lightgbm을 wrapping한 것
from sklearn.model_selection import train_test_split
from pandas.tseries.offsets import CustomBusinessDay
import datetime
from sklearn.metrics import root_mean_squared_error
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import Utility as Util

file_path = r"/home/ymatics/CodingSpace/2024_AI_BEMS/df_raw.pickle"
df_raw = pd.read_pickle(file_path)
print(f"timestamp: {df_raw.index[0]}, {df_raw.index[-1]}")
#start_date, end_date = pd.to_datetime('2024-04-01 00:00:00'), pd.to_datetime('2024-07-01 13:00:00')
#df = df_raw.loc[start_date:end_date].copy()
df = df_raw.copy()
Util.plot_data(df, plotType='simple', title=None, W=10, H=5)

In [ ]:
p_ = 4  # 시간당 4 points 샘플링, 15min 간격
freq = '15min'

# 1. 누락여부 피쳐 생성 및 보간
df.index = df.index.round(freq)  # freq 단위로 라운딩하여 시간 맞추고(15분+=50% time index 흔들림 허용)
df = df[~df.index.duplicated(keep='first')]  # 중복된 시간 포인트는 제거
df = df.resample(freq).asfreq()  # freq 단위로 인덱스 리샘플링, asfreq()로 빈 시간대는 NaN으로 채움
df['is_missing'] = df['target'].isna().astype(int)
df.ffill(inplace=True)  # 예측 모델인 경우, 미래 데이터를 사용하지 않도록 보간
df.bfill(inplace=True)  # 예측 모델인 경우, 만약 첫 번째 값이 결측치인 경우를 대비
#df['target'] = df['target'].interpolate(method='time')  # 회귀 모델인 경우, 대안으로 선택 가능

df_shift_1p = df['target'].shift(1)  # Shifted targets to use past data only

# 2. 시간 기반 피쳐 생성 (대한민국의 주말 및 공휴일 특징 반영)
kr_holidays = holidays.KR()
kr_business_day = CustomBusinessDay(holidays=kr_holidays)
df['month'] = df.index.month
df['day'] = df.index.day
df['weekday'] = df.index.weekday
df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)  # 주말 여부
df['is_holiday'] = df.index.isin(kr_holidays).astype(int)  # 대한민국 공휴일 여부

# 2. 명절 연휴 처리(설날, 추석은 전일과 후일이 연휴임)
major_holiday_dates = pd.to_datetime([date for date in kr_holidays if kr_holidays[date] in ['설날', '추석']])
df.loc[df.index.isin(major_holiday_dates - pd.DateOffset(days=1)), 'is_holiday'] = 1
df.loc[df.index.isin(major_holiday_dates + pd.DateOffset(days=1)), 'is_holiday'] = 1

# 2. 대체공휴일 보정
holiday_dates = pd.to_datetime(list(kr_holidays.keys()))  # kr_holidays를 datetime으로 변환
for date in kr_holidays:
    if date.weekday() in [5, 6]:
        replacement_date = date + datetime.timedelta(days=1)
        while replacement_date.weekday() in [5, 6] or replacement_date in kr_holidays:
            replacement_date += datetime.timedelta(days=1)
        df.loc[replacement_date, 'is_holiday'] = 1

# 3. 시간 지연 피쳐 생성
df['lag_1p'] = df['target'].shift(1)  # 1h delayed pattern
df['lag_2p'] = df['target'].shift(2)  
df['lag_3p'] = df['target'].shift(3)  
df['lag_4p'] = df['target'].shift(4)  
df['lag_1d_0p'] = df['target'].shift(p_ * 24 + 0)  # 1day = 4p_ x 24h/p_ delayed pattern
df['lag_1d_1p'] = df['target'].shift(p_ * 24 + 1)  
df['lag_1d_2p'] = df['target'].shift(p_ * 24 + 2)  
df['lag_1d_3p'] = df['target'].shift(p_ * 24 + 3)  
df['lag_1d_4p'] = df['target'].shift(p_ * 24 + 4)  
df['lag_1w_0p'] = df['target'].shift(p_ * 24 * 7 + 0)  # 1w = 4p_ x 24h/p_ x 7days delayed pattern
df['lag_1w_1p'] = df['target'].shift(p_ * 24 * 7 + 1)  
df['lag_1w_2p'] = df['target'].shift(p_ * 24 * 7 + 2)  
df['lag_1w_3p'] = df['target'].shift(p_ * 24 * 7 + 3)  
df['lag_1w_4p'] = df['target'].shift(p_ * 24 * 7 + 4) 

# 4. 변동률 및 변동률의 변동률 피쳐 생성
epsilon = 1e-3
shifted_1p = df['target'].shift(1)  # Shifted targets to use past data only
divisor_1p = np.where(np.abs(shifted_1p) > epsilon, shifted_1p, np.sign(shifted_1p) * epsilon)
df['rate'] = (shifted_1p - shifted_1p.shift(1)) / divisor_1p
df['diff_rate'] = df['rate'].diff()

shifted_1d = df['target'].shift(p_ * 24)  # 하루 전의 동일 시간대 변동률의 변동률
divisor_1d = np.where(np.abs(shifted_1d) > epsilon, shifted_1d, np.sign(shifted_1d) * epsilon)
df['rate_1d'] = (shifted_1d - shifted_1d.shift(1)) / divisor_1d
df['diff_rate_1d'] = df['rate_1d'].diff()

# 5. 윈도우 통계 피처 생성
df['ma_1h'] = df_shift_1p.rolling(window=p_).mean()  # 1시간 통계, shift(1) 적용으로 과거 데이터 사용
df['max_1h'] = df_shift_1p.rolling(window=p_).max()  
df['min_1h'] = df_shift_1p.rolling(window=p_).min()  
df['std_1h'] = df_shift_1p.rolling(window=p_).std() 

df['ma_1d'] = df_shift_1p.rolling(window=p_ * 24).mean()  # 1일 통계
df['max_1d'] = df_shift_1p.rolling(window=p_ * 24).max()  
df['min_1d'] = df_shift_1p.rolling(window=p_ * 24).min()  
df['std_1d'] = df_shift_1p.rolling(window=p_ * 24).std() 

df['p1d_ma_1d'] = df['target'].shift(p_*24).rolling(window=p_ * 24).mean()  # 1일전 동시간대 1day 통계
df['p1d_max_1d'] = df['target'].shift(p_*24).rolling(window=p_ * 24).max()  
df['p1d_min_1d'] = df['target'].shift(p_*24).rolling(window=p_ * 24).min()  
df['p1d_std_1d'] = df['target'].shift(p_*24).rolling(window=p_ * 24).std() 

df['p1w_ma_1d'] = df['target'].shift(p_*24*7).rolling(window=p_ * 24).mean()  # 1주일전 동시간대 1day 통계
df['p1w_max_1d'] = df['target'].shift(p_*24*7).rolling(window=p_ * 24).max()  
df['p1w_min_1d'] = df['target'].shift(p_*24*7).rolling(window=p_ * 24).min()  
df['p1w_std_1d'] = df['target'].shift(p_*24*7).rolling(window=p_ * 24).std() 

# 6. 이동 평균의 변화율 특징 생성
epsilon = 1e-3
shifted_1p = df['ma_1h']
divisor_1p = np.where(np.abs(shifted_1p) > epsilon, shifted_1p, np.sign(shifted_1p) * epsilon)
df['rate_ma_1h'] = (shifted_1p - shifted_1p.shift(1)) / divisor_1p

shifted_1p = df['ma_1d']
divisor_1p = np.where(np.abs(shifted_1p) > epsilon, shifted_1p, np.sign(shifted_1p) * epsilon)
df['rate_ma_1d'] = (shifted_1p - shifted_1p.shift(1)) / divisor_1p

# 7. 추세 및 계절성 분포
df['trend'] = np.arange(len(df))
df['season'] = df_shift_1p - df['ma_1d']  # shift(1) 적용으로 과거 데이터 사용

# 8. 계절성 피처 생성
df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)
df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12)

# 9. 주기성 특성 (Cyclical Features)
df['sine_day'] = np.sin(2 * np.pi * df.index.dayofyear / 365.25)
df['cosine_day'] = np.cos(2 * np.pi * df.index.dayofyear / 365.25)

# 10. 이상치 여부를 나타내는 특징 생성
Q1 = df_shift_1p.rolling(window=p_*24).quantile(0.25)
Q3 = df_shift_1p.rolling(window=p_*24).quantile(0.75)
IQR = Q3 - Q1
df['is_outlier'] = ((df_shift_1p < (Q1 - 1.5 * IQR)) | (df_shift_1p > (Q3 + 1.5 * IQR))).astype(int)  # 이상치 여부

# 11. Fourier Transform 특징 생성
fft_features = fft(df_shift_1p.fillna(0).values)
df['fft_real'] = np.real(fft_features)  # 실수 부분
df['fft_imag'] = np.imag(fft_features)  # 허수 부분

# 12. 추가적인 계절성 특징 생성
df['quarter'] = df.index.quarter  # 분기
df['hour'] = df.index.hour        # 시간대
df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 24)
df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 24)

# 12. 외부 데이터 통합 (예시: 기온 데이터)
# 예를 들어, 'temperature.csv' 파일에 날짜별 기온 데이터가 있다고 가정
# temperature_df = pd.read_csv('temperature.csv', index_col='date', parse_dates=True)
# df = df.join(temperature_df, on='date')  # 기온 데이터 통합

# 13. 지수 이동 평균 (Exponential Moving Average)
df['ema_1d'] = df_shift_1p.ewm(span=p_*24, adjust=False).mean()  # 이전 1일 지수 이동 평균

# 14. 엔트로피 특징 생성
def calc_entropy(x):
    counts = np.histogram(x.dropna(), bins=10)[0]
    return entropy(counts)
df['entropy_1d'] = df_shift_1p.rolling(window=p_*24).apply(calc_entropy)  # 1일 엔트로피

# 15. 누적 합계 및 변화율
df['cum_sum'] = df_shift_1p.cumsum()  # 누적 합계
df['cum_pct'] = df['cum_sum'].pct_change()  # 누적 합계의 증감율

nan_counts = df.isna().sum()  # 각 열마다 NaN의 갯수 출력
for index, item in enumerate(nan_counts):
    print(f"{df.columns[index]= }, NaN Count: {item}")

# 결측값 처리 (피처 생성시 시간 지연으로 인해 발생)
df.dropna(inplace=True)

# 피처벡터와 타겟 분리
X = df.drop('target', axis=1)
y = df['target']

### Step B. lightGBM Modeling
- train_test_split() 에서, prediction model에서는 shuffle=False, regression model에서는 shuffle=True

In [ ]:
# 학습 및 테스트 데이터 분할
# 시계열 데이터는 시간 순으로 되어 있어야 하고, shuffle=False로 순방향 데이터검증 보장
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
#train_size = int(len(X) * 0.8)
#X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
#y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

# LightGBM 데이터 세트 생성
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# 모델 하이퍼파라미터 설정
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 30,
    'verbose': -1
}

# 모델 학습
model_a = lgb.train(params,
                    train_data,
                    valid_sets=[valid_data],
                    num_boost_round=1000,
                    valid_names=['validation'],
                    callbacks=[lgb.early_stopping(stopping_rounds=30)])

##################
# 예측 수행
y_pred_a = model_a.predict(X_test, num_iteration=model_a.best_iteration)

# RMSE 출력
rmse_a = root_mean_squared_error(y_test, y_pred_a)
print(f'Experiment A: {rmse_a= :.2f} with {X.shape[1]} features')

# 예측 결과 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['target'], mode='lines', name='Actual Target'))
fig.add_trace(go.Scatter(x=y_test.index, y=y_test, mode='lines', name='Actual Test'))
fig.add_trace(go.Scatter(x=y_test.index, y=y_pred_a, mode='lines', name='Predicted'))

# Update layout
fig.update_layout(title='Actual vs Predicted Values',
                  xaxis_title='Date',
                  yaxis_title='Values',
                  legend_title='Legend')
fig.show()

# Feature Importance Visualization
# 모델의 특성 중요도 추출
feature_importances = model_a.feature_importance()
feature_names = X.columns

# 특성 중요도를 데이터프레임으로 정리
importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
importance_df = importance_df.sort_values(by='importance', ascending=False)

# 특성 중요도 시각화
plt.figure(figsize=(10, 12))  # 그래프 크기 설정
plt.barh(importance_df['feature'], importance_df['importance'])  # 수평 막대그래프 생성
plt.xlabel('Feature Importance')  # x축 레이블 설정
plt.title('Feature Importance Visualization')  # 그래프 제목 설정
plt.gca().invert_yaxis()  # 중요도가 높은 순으로 표시
plt.show()  # 그래프 출력

### Step C. Features Reduction

In [ ]:
# Experiment B. Feature Reduction을 적용 후 (Embedded Method with lightgbm 적용)

from sklearn.feature_selection import SelectFromModel

# 기존 모델과 동일한 하이퍼파라미터 설정
lgb_estimator = lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    boosting_type='gbdt',
    learning_rate=0.1,
    num_leaves=30,
    verbose=-1
)

# SelectFromModel을 사용하여 중요하지 않은 특성 제거
selector = SelectFromModel(estimator=lgb_estimator, threshold='median')

# 훈련 데이터에 대해 fit
selector.fit(X_train, y_train)

# 선택된 특성 이름 추출
selected_features = X_train.columns[selector.get_support()]

# 선택된 특성으로 데이터셋 변환
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# LightGBM 데이터 세트 생성 (선택된 특성 사용)
train_data_selected = lgb.Dataset(X_train_selected, label=y_train)
valid_data_selected = lgb.Dataset(X_test_selected, label=y_test, reference=train_data_selected)

# 모델 재학습 (선택된 특성 사용)
model_b = lgb.train(params,
                    train_data_selected,
                    valid_sets=[valid_data_selected],
                    num_boost_round=1000,
                    valid_names=['validation'],
                    callbacks=[lgb.early_stopping(stopping_rounds=30)])

# 예측 수행
y_pred_b = model_b.predict(X_test_selected, num_iteration=model_b.best_iteration)

# RMSE 출력
rmse_b = root_mean_squared_error(y_test, y_pred_b)
print(f'Experiment A: {rmse_a= :.2f} with {X.shape[1]} features')
print(f'Experiment B: {rmse_b= :.2f} with {X_train_selected.shape[1]} features')

# 선택된 특성 중요도 시각화
feature_importances_b = model_b.feature_importance()
importance_df_b = pd.DataFrame({'feature': selected_features, 'importance': feature_importances_b})
importance_df_b = importance_df_b.sort_values(by='importance', ascending=False)

plt.figure(figsize=(10, 10))
plt.barh(importance_df_b['feature'], importance_df_b['importance'])
plt.xlabel('Feature Importance')
plt.title('Feature Importance after Embedded Method')
plt.gca().invert_yaxis()
plt.show()

### Step D. Hyperparameter Optimization

In [ ]:
import optuna

# 학습 및 테스트 데이터 분할
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
X_train, X_test, y_train, y_test = X_train_selected, X_test_selected, y_train, y_test

# LightGBM 데이터 세트 생성
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# 모델 하이퍼파라미터 설정 및 optuna를 이용한 튜닝
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'verbose': -1
    }
    
    # 모델 학습
    model = lgb.train(params,
                      train_data,
                      valid_sets=[valid_data],
                      num_boost_round=1000,
                      valid_names=['validation'],
                      callbacks=[lgb.early_stopping(stopping_rounds=30)])
    
    # 예측 수행
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    
    # RMSE 계산
    rmse = root_mean_squared_error(y_test, y_pred)
    return rmse

# 최적화 수행
# Optuna 스터디 생성 (SQLite 스토리지 사용)
STUDY_NAME = "optuna_study"
DB_PATH = "sqlite:///optuna_study.db"  # SQLite 데이터베이스 파일 경로
study = optuna.create_study(
    study_name=STUDY_NAME,
    storage=DB_PATH,
    direction='minimize',
    load_if_exists=True  # 기존 스터디가 있으면 불러옴
)
study.optimize(objective, n_trials=10)
print('Best hyperparameters: ', study.best_params)

# 최적 하이퍼파라미터로 모델 재학습
best_params = study.best_params
best_params['objective'] = 'regression'
best_params['metric'] = 'rmse'
best_params['boosting_type'] = 'gbdt'

model_opt = lgb.train(best_params,
                    train_data,
                    valid_sets=[valid_data],
                    num_boost_round=1000,
                    valid_names=['validation'],
                    callbacks=[lgb.early_stopping(stopping_rounds=50)])

# 예측 수행
y_pred_opt = model_opt.predict(X_test, num_iteration=model_opt.best_iteration)

# RMSE 출력
rmse_opt = root_mean_squared_error(y_test, y_pred_opt)

print('*'*30) 
print(f'Experiment(Optimization): {rmse_opt= :.2f} with {X_train_selected.shape[1]} features')
print(f'{model_opt.params= }')

### Step E: Save the model

In [ ]:
model_a.save_model('lightgbm_model_a.txt')
model_b.save_model('lightgbm_model_b.txt')
model_opt.save_model('lightgbm_model_opt.txt')

# 모델 로드
model_opt_infer = lgb.Booster(model_file='lightgbm_model_opt.txt')

# 예측 수행 (로드된 모델 사용)
y_pred_loaded = model_opt_infer.predict(X_test)

# 모델 평가
rmse_loaded = root_mean_squared_error(y_test, y_pred_loaded)
print(f'RMSE (Loaded Model): {rmse_loaded= :.3f}')

# 결과 시각화
plt.figure(figsize=(10, 3))
plt.plot(y_test.index, y_test, label='Actual')
plt.plot(y_test.index, y_pred_loaded, label='Predicted (Loaded Model)', color='orange')
plt.legend()
plt.title('LightGBM Time Series Prediction (Loaded Model)')
plt.show()


### Coding Test

In [ ]:
df

### LightGBM prediction: with missing intervals

In [ ]:
import pandas as pd
import Utility as Util

file_path = r"/home/ymatics/CodingSpace/2024_AI_BEMS/df_raw.pickle"
df_raw = pd.read_pickle(file_path)
print(f"timestamp: {df_raw.index[0]}, {df_raw.index[-1]}")
#start_date, end_date = pd.to_datetime('2024-04-01 00:00:00'), pd.to_datetime('2024-07-01 13:00:00')
#df = df_raw.loc[start_date:end_date].copy()
df = df_raw.copy()
Util.plot_data(df, plotType='simple', title=None, W=10, H=5)

#df_interpol = Util.resample_time_index_interpolate_NaN_df(df_raw, '15min', 'ffill')

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# 1. 예제 데이터 생성 (랜덤하게 구간 누락)
data_o = df.copy()

# 2. 일부 구간 누락 생성 함수

# 3. 누락 구간 설정

# 4. 데이터에서 누락 구간 제거

# 5. 데이터 복사
data = data_o.copy()

# 6. 누락 여부 피처 생성 및 보간
data['is_missing'] = data['target'].isna().astype(int)

data['target'] = data['target'].interpolate(method='time')
#data.ffill(inplace=True)  # 미래 데이터를 사용하지 않도록 보간 방법 변경
#data.bfill(inplace=True)  # 만약 첫 번째 값이 결측치인 경우를 대비

# 7. 시간 관련 피처 생성
data['hour'] = data.index.hour
data['dayofweek'] = data.index.dayofweek
data['month'] = data.index.month

# 8. 시차 피처 생성
data['lag_1'] = data['target'].shift(1)
data['lag_24'] = data['target'].shift(24)

# 9. 이동 평균 및 표준편차 피처 생성
data['rolling_mean_3'] = data['target'].rolling(window=3).mean()
data['rolling_std_3'] = data['target'].rolling(window=3).std()

# 10. 결측치 제거 (피처 생성 후)
data.dropna(inplace=True)

# 11. 학습 및 테스트 데이터 분할
train_data = data.loc['2023-01-01':'2024-06-01']
test_data = data.loc['2024-06-02':'2024-10-14']

X_train = train_data.drop('target', axis=1)
y_train = train_data['target']
X_test = test_data.drop('target', axis=1)
y_test = test_data['target']

# 12. LightGBM 데이터셋 생성
train_dataset = lgb.Dataset(X_train, label=y_train)
test_dataset = lgb.Dataset(X_test, label=y_test, reference=train_dataset)

# 13. 모델 학습
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1
}

model = lgb.train(params, train_dataset, valid_sets=[test_dataset], callbacks=[lgb.early_stopping(stopping_rounds=10)])

# 14. 예측 및 평가
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse}')

# 15. 데이터 통합 및 시각화 준비
# y_test와 y_pred를 전체 시간 구간에 맞춰서 재구성
full_dates = data_o.index

# Create a DataFrame to hold all series
plot_df = pd.DataFrame(index=full_dates)
plot_df['Original Data'] = data_o['target']
plot_df['Interpolated Data'] = data['target']

# y_test and y_pred need to be reindexed to full date range
y_test_full = pd.Series(data=y_test.values, index=y_test.index)
y_test_full = y_test_full.reindex(full_dates)
plot_df['Test Data Actual'] = y_test_full

y_pred_full = pd.Series(data=y_pred, index=y_test.index)
y_pred_full = y_pred_full.reindex(full_dates)
plot_df['Test Data Predicted'] = y_pred_full

# 16. Plotly를 이용한 시각화
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['Original Data'], mode='lines', name='Original Data', line=dict(color='orange', width=1)))
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['Interpolated Data'], mode='lines', name='Interpolated Data', line=dict(color='blue', width=1)))
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['Test Data Actual'], mode='lines', name='Test Data Actual', line=dict(color='green', width=2)))
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['Test Data Predicted'], mode='lines', name='Test Data Predicted', line=dict(color='red', width=2, dash='dot')))
fig.update_layout(
    title='Time Series Data Visualization',
    xaxis_title='Date',
    yaxis_title='Value',
    legend=dict(x=0.01, y=0.99),
)
fig.show()

# 모델 훈련 후 feature importance 시각화
lgb.plot_importance(model, max_num_features=20)  # 상위 20개의 중요 특징만 표시
plt.title("Feature Importance")
plt.show()
